In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# export
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# export
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks/exp")
from nb_03 import *
import torch.nn.functional as F

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Databunch/Learner

In [0]:
# Downloading data
x_train, y_train, x_valid, y_valid = get_data()

# Creating dataset
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh, bs = 50, 64

c = y_train.max().item() + 1
loss_func = F.cross_entropy

Factor out the connected pieces of info out of the `fit()` argument list

`fit(epochs, model, loss_func, opt, train_dl, valid_dl)`

Let's replace it with something that looks like this:

`fit(1, learn)`

This will allow us to tweak what's happening inside the training loop in other places of the code because the `Learner` object will be mutable, so changing any of its attribute elsewhere will be seen in our training loop.

In [0]:
# export
class Databunch():
  def __init__(self, train_dl, valid_dl, c = None):
    self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c

  @property
  def train_ds(self):
    return self.train_dl.dataset

  @property
  def valid_ds(self):
    return self.valid_dl.dataset


In [0]:
data = Databunch(*get_dls(train_ds, valid_ds, bs), c)

In [0]:
# export
def get_model(data, lr=0.5, nh=50):
  m = data.train_ds.x.shape[1]
  model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, data.c))
  return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
  def __init__(self, model, opt, loss_func, data):
    self.model, self.opt, self.loss_func, self.data = model, opt, loss_func, data

In [0]:
learn = Learner(*get_model(data), loss_func=loss_func, data=data)

In [0]:
def fit(epochs, learn):
  for epoch in range(epochs):
    learn.model.train()
    for xb, yb in learn.data.train_dl:
      loss = learn.loss_func(learn.model(xb), yb)
      loss.backward()
      learn.opt.step()
      learn.opt.zero_grad()

    learn.model.eval()
    with torch.no_grad():
      tot_loss, tot_acc = 0., 0.
      for xb, yb in learn.data.valid_dl:
        pred = learn.model(xb)
        tot_loss += loss_func(pred, yb)
        tot_acc += accuracy(pred, yb)
      nv = len(learn.data.valid_dl)
      print(f"Epoch:{epoch}, Total loss: {tot_loss/nv}, Total accuracy:{tot_acc/nv}")
  return tot_loss/nv, tot_acc/nv   

In [10]:
loss, acc = fit(10, learn)

Epoch:0, Total loss: 0.1994941383600235, Total accuracy:0.940386176109314
Epoch:1, Total loss: 0.20250773429870605, Total accuracy:0.9387937784194946
Epoch:2, Total loss: 0.13575085997581482, Total accuracy:0.956707775592804
Epoch:3, Total loss: 0.11080797761678696, Total accuracy:0.9661624431610107
Epoch:4, Total loss: 0.21162395179271698, Total accuracy:0.9367038011550903
Epoch:5, Total loss: 0.09289976209402084, Total accuracy:0.9722332954406738
Epoch:6, Total loss: 0.0916488990187645, Total accuracy:0.9753184914588928
Epoch:7, Total loss: 0.10028642416000366, Total accuracy:0.9733280539512634
Epoch:8, Total loss: 0.09754765033721924, Total accuracy:0.9744227528572083
Epoch:9, Total loss: 0.11375395953655243, Total accuracy:0.9701433181762695


## Callback Handler
This was our training loop (without validation) from the previous notebook, with the inner loop contents factored out:
```
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()

def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch(*b)

```



Add callbacks so we can remove complexity from loop, and make it flexible:

In [0]:
class Callback():
  def begin_fit(self, learn):
    self.learn = learn
    return True

  def after_fit(self): return True

  def begin_epoch(self, epoch): 
    self.epoch = epoch
    return True

  def begin_validate(self): return True

  def after_epoch(self): return True

  def begin_batch(self, xb, yb):
    self.xb, self.yb = xb, yb
    return True

  def after_loss(self,loss):
    self.loss = loss
    return True

  def after_backward(self): return True

  def after_step(self): return True

In [0]:
class CallbackHandler():
  def __init__(self, cbs=None):
    self.cbs = cbs if cbs else []

  def begin_fit(self, learn):
    self.learn, self.in_train = learn, True
    learn.stop = False
    res = True
    for cb in self.cbs:
      res = res and cb.begin_fit(learn)
    return res

  def after_fit(self):
    res = not self.in_train
    for cb in self.cbs:
      res = res and cb.after_fit()
    return True

  def begin_epoch(self, epoch):
    self.learn.model.train()
    self.in_train = True
    res = True
    for cb in self.cbs:
      res = res and cb.begin_epoch(epoch)
    return res

  def begin_validate(self):
    self.learn.model.eval()
    self.in_train = False
    res = True
    for cb in self.cbs:
      res = res and cb.begin_validate()
    return res

  def after_epoch(self):
    res = True
    for cb in self.cbs:
      res = res and cb.after_epoch()
    return res

  def begin_batch(self, xb, yb):
    res = True
    for cb in self.cbs:
      res = res and cb.begin_batch(xb, yb)
    return res

  def after_loss(self, loss):
    res = self.in_train
    for cb in self.cbs:
      res = res and cb.after_loss(loss)
    return res

  def after_backward(self):
    res = True
    for cb in self.cbs:
      res = res and cb.after_backward()
    return res

  def after_step(self):
    res = True
    for cb in self.cbs:
      res = res and cb.after_step()
    return res

  def do_stop(self):
    try:
      return self.learn.stop
    finally:
      self.learn.stop = False

In [0]:
def one_batch(xb, yb, cb):
  if not cb.begin_batch(xb, yb): return
  loss = cb.learn.loss_func(cb.learn.model(xb), yb)
  if not cb.after_loss(loss): return
  if cb.after_backward(): cb.learn.opt.step()
  if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
  for xb, yb in dl:
    one_batch(xb, yb, cb)
    if cb.do_stop(): return

def fit(epochs, learn, cb):
  if not cb.begin_fit(learn): return
  for epoch in range(epochs):
    if not cb.begin_epoch(epoch): continue
    all_batches(learn.data.train_dl, cb)

    if cb.begin_validate():
      with torch.no_grad():
        all_batches(learn.data.valid_dl, cb)
    
    if cb.do_stop() or not cb.after_epoch():
      break
    cb.after_fit()

In [0]:
class TestCallback(Callback):
  def begin_fit(self, learn):
    super().begin_fit(learn)
    self.n_iters = 0
    return True

  def after_step(self):
    self.n_iters += 1
    print(self.n_iters)
    if self.n_iters >= 10:
      self.learn.stop = True
    return True

  def begin_epoch(self, epoch):
    self.n_iters = 0
    print(f'Epoch {epoch}')
    return True

In [15]:
fit(3, learn, cb=CallbackHandler([TestCallback()]))

Epoch 0
1
2
3
4
5
6
7
8
9
10
Epoch 1
1
2
3
4
5
6
7
8
9
10
Epoch 2
1
2
3
4
5
6
7
8
9
10


This is roughly how fastai does it now (except that the handler can also change and return xb, yb, and loss). But let's see if we can make things simpler and more flexible, so that a single class has access to everything and can change anything at any time. The fact that we're passing cb to so many functions is a strong hint they should all be in the same class!

## Runner

In [0]:
# export
import re

_camel_re1 = re.compile('(.+?)([A-Z])')
_camel_re2 = re.compile('(.*?)_([a-z0-9A-Z])')

def camel2snake(name):
  s1 = re.sub(_camel_re1, r'\1_\2', name)
  return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
  _order = 0
  def set_runner(self, run):
    self.run = run

  def __getattr__(self, k):
    return getattr(self.run, k)

  @property
  def name(self):
    name = re.sub(r'Callback$','',self.__class__.__name__)
    return camel2snake(name or 'callback')

This first callback is reponsible to switch the model back and forth in training or validation mode, as well as maintaining a count of the iterations, or the percentage of iterations ellapsed in the epoch.

In [0]:
# export
class TrainEvalCallback(Callback):
  def begin_fit(self):
    self.run.n_epochs = 0.
    self.run.n_iter = 0

  def after_batch(self):
    if not self.in_train: return
    self.run.n_epochs += 1./self.iter
    self.run.n_iter += 1

  def begin_epoch(self):
    self.run.n_epochs = self.epoch
    self.model.train()
    self.run.in_train=True

  def begin_validate(self):
    self.model.eval()
    self.run.in_train=False

We'll also re-create our TestCallback

In [0]:
class TestCallback(Callback):
  def after_step(self):
    if self.train_eval.n_iters >= 10:
      return True

In [19]:
cbname = 'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [20]:
TrainEvalCallback().name

'train_eval'

In [0]:
# export

from typing import *
def listify(o):
  if o is None: return []
  if isinstance(o, list): return o
  if isinstance(o, str): return [o]
  if isinstance(o, Iterable): return list(o)
  return [o]

In [0]:
# export
class Runner():
  def __init__(self, cbs = None, cb_funcs = None):
    cbs = listify(cbs)
    for cbf in listify(cb_funcs):
      cb = cbf()
      setattr(self, cb.name, cb)
      cbs.append(cb)
    self.stop, self.cbs = False, [TrainEvalCallback()]+cbs

  @property
  def opt(self):
    return self.learn.opt

  @property
  def model(self):
    return self.learn.model

  @property
  def loss_func(self):
    return self.learn.loss_func

  @property
  def data(self):
    return self.learn.data

  def one_batch(self, xb, yb):
    self.xb, self.yb = xb, yb
    if self('begin_batch'): return
    self.pred = self.model(self.xb)
    
    if self('after_pred'): return
    self.loss = self.loss_func(self.pred, self.yb)

    if self('after_loss') or not self.in_train: return
    
    self.loss.backward()

    if self('after_backward'): return
    self.opt.step()

    if self('after_step'): return
    self.opt.zero_grad()

  def all_batches(self, dl):
    self.iter = len(dl)
    for xb, yb in dl:
      if self.stop: break
      self.one_batch(xb, yb)
      self('after_batch')
    self.stop = False

  def fit(self, epochs, learn):
    self.epochs, self.learn = epochs, learn

    try:
      for cb in self.cbs: cb.set_runner(self)
      if self('begin_fit'): return
      for epoch in range(epochs):
        self.epoch = epoch
        if not self('begin_epoch'): 
          self.all_batches(self.data.train_dl)

        with torch.no_grad():
          if not self('begin_validate'):
            self.all_batches(self.data.valid_dl)
          if self('after_epoch'): break

    finally:
      self('after_fit')
      self.learn = None

  def __call__(self, cb_name):
    for cb in sorted(self.cbs, key=lambda x: x._order):
      f = getattr(cb, cb_name, None)
      if f and f(): return True
    return False



Third callback: how to compute metrics.

In [0]:
# export
class AvgStats():
  def __init__(self, metrics, in_train):
    self.metrics, self.in_train = listify(metrics), in_train

  def reset(self):
    self.tot_loss, self.count = 0.,0
    self.tot_mets = [0.] * len(self.metrics)

  @property
  def all_stats(self):
    return [self.tot_loss.item()] + self.tot_mets

  @property
  def avg_stats(self):
    return[o/self.count for o in self.all_stats]

  def __repr__(self):
    if not self.count: return ""
    return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

  def accumulate(self, run):
    bn = run.xb.shape[0]
    self.tot_loss += run.loss * bn
    self.count += bn
    for i,m in enumerate(self.metrics):
      self.tot_mets[i] += m(run.pred, run.yb) * bn

class AvgStatsCallback(Callback):
  def __init__(self, metrics):
    self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)      

  def begin_epoch(self):
    self.train_stats.reset()
    self.valid_stats.reset()

  def after_loss(self):
    stats = self.train_stats if self.in_train else self.valid_stats
    with torch.no_grad():
      stats.accumulate(self.run)

  def after_epoch(self):
    print(self.train_stats)
    print(self.valid_stats)

In [0]:
learn = Learner(*get_model(data), loss_func, data)

In [0]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [36]:
run.fit(2, learn)

train: [0.316200078125, tensor(0.9031)]
valid: [0.17605506591796874, tensor(0.9446)]
train: [0.14369666015625, tensor(0.9572)]
valid: [0.133343798828125, tensor(0.9617)]


In [40]:
loss, acc = stats.valid_stats.avg_stats
assert acc > 0.9
loss, acc

(0.133343798828125, tensor(0.9617))

In [0]:
# export
from functools import partial

In [0]:
acc_cbf = partial(AvgStatsCallback, accuracy)

In [0]:
run = Runner(cb_funcs=acc_cbf)

In [51]:
run.fit(1, learn)

train: [0.111734794921875, tensor(0.9659)]
valid: [0.1094269287109375, tensor(0.9669)]


In [52]:
run.avg_stats.valid_stats.avg_stats

[0.1094269287109375, tensor(0.9669)]

##Export

In [53]:
!pip install fire

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=4b0a57f4b5f26d0d87903007ad58a31a2291729fc94d1f412e2ded35d3aad6f6
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire


In [54]:
!python "/content/drive/My Drive/Colab Notebooks/notebook2script.py" "/content/drive/My Drive/Colab Notebooks/04_callbacks.ipynb"

Converted /content/drive/My Drive/Colab Notebooks/04_callbacks.ipynb to /content/drive/My Drive/Colab Notebooks/exp/nb_04.py
